# Notebook to develop different objective functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

In [ ]:
def tau(k,A,c): 
    #generates tau vector 
    return 1/ np.math.factorial(k)*c**k - 1/np.math.factorial(k-1) *A @ c**(k-1)
    
def tau_hat(k,A,c):
    return c**k-k*A@(k-1)


def OrderCond(A,c,order = 1):
    #Generates Order Condition Matrix O and right side vector r for Linear Equation System O@b=r
    
    s = len(c) #number of stages
    
    r = []
    O_rows = []
    
    
    if A.shape != (s,s):
        raise InputError
        
    else:
        if order >= 1:
            O_rows.append(np.ones(s));      r.append(1)
            
        if order >= 2:
            O_rows.append(c);               r.append(1/2)
            
        if order >= 3:
            O_rows.append(c**2);            r.append(1/3)
            O_rows.append(tau(2,A,c));      r.append(0.)
            
        if order >= 4:
            O_rows.append(c**3);            r.append(1/4)
            O_rows.append(tau(2,A,c)*c);    r.append(0.)
            O_rows.append(tau(2,A,c)@A.T);  r.append(0.)
            O_rows.append(tau(3,A,c));      r.append(0.)
        
        if order >= 5:
            O_rows.append(c**4);                     r.append(1/5)
            O_rows.append(A@np.diag(c)@tau(2,A,c));  r.append(0.)
            O_rows.append(A@A@tau(2,A,c));           r.append(0.)
            O_rows.append(A@tau(3,A,c));             r.append(0.)
            O_rows.append(tau(4,A,c));               r.append(0.)
            O_rows.append(np.diag(c)@A@tau(2,A,c));  r.append(0.)
            O_rows.append(np.diag(c)@tau(3,A,c));    r.append(0.)
            O_rows.append(np.diag(c**2)@tau(2,A,c)); r.append(0.)
            O_rows.append(tau(2,A,c)**2);            r.append(0.)
        if order >= 6:
            print('too high order')
            raise NotImplementedError
        
        O = np.vstack(O_rows)
        return (O,np.array(r))
            
                
    

In [ ]:
# Quadrature condition for Objective Function

def QuadCond(A,c,order =2):
    s = len(c)
    if A.shape != (s,s):
        raise InputError    
        
    if order == 1:
        o = np.ones(s);                   r=1.
    elif order == 2:
        o = c;                            r=1/2
    elif order == 3:
        o = c**2;                         r=1/3
    elif order == 4:
        o = c**3;                         r=1/4
    elif order == 5:
        o = c**4;                         r=1/5
    else:
        o = c**(order-1);    r = 1/order
    return(o,r) 
        
    
def OrderOpt(A,c,order =2):

    s = len(c) #number of stages
    
    r = []
    O_rows = []
    
    
    if A.shape != (s,s):
        raise InputError
        
    else:
        if order == 1:
            raise InputError
            
        if order == 2:
            raise InputError
            #O_rows.append(c);               r.append(1/2)
            
        if order == 3:
            #O_rows.append(c**2);            r.append(1/3)
            O_rows.append(tau(2,A,c));      r.append(0.)
            
        if order == 4:
            #O_rows.append(c**3);            r.append(1/4)
            O_rows.append(tau(2,A,c)*c);    r.append(0.)
            O_rows.append(tau(2,A,c)@A.T);  r.append(0.)
            O_rows.append(tau(3,A,c));      r.append(0.)
        
        if order == 5:
            #O_rows.append(c**4);                     r.append(1/5)
            O_rows.append(A@np.diag(c)@tau(2,A,c));  r.append(0.)
            O_rows.append(A@A@tau(2,A,c));           r.append(0.)
            O_rows.append(A@tau(3,A,c));             r.append(0.)
            O_rows.append(tau(4,A,c));               r.append(0.)
            O_rows.append(np.diag(c)@A@tau(2,A,c));  r.append(0.)
            O_rows.append(np.diag(c)@tau(3,A,c));    r.append(0.)
            O_rows.append(np.diag(c**2)@tau(2,A,c)); r.append(0.)
            O_rows.append(tau(2,A,c)**2);            r.append(0.)
        if order >= 6:
            print('too high order')
            raise NotImplementedError
        
        O = np.vstack(O_rows)
        return (O,np.array(r))
    

In [ ]:
def RK_variable_b(rkm, dt, f, w0=[1.,0], t_final=1.,b_fixed = False,objective = 'L1 of b',solver = cp.ECOS):
    """    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time    
        
        Modes for opjective:
            'L1 of b':      The optimisation Problem opimises for min(|b-rkm.b|_1)
            
            'Quadrature':   The optimisation Problem opimises for min(|bTq-r|) 
                            where q is the quadrature condition of the next higher order 
                            and r is the expected right hand side
            
            'Hom. Order':   The optimisation Problem opimises for min(|bTO|) 
                            where O are the homogenus Order Conditions of the next higher orders
                            
            'Order':        The optimisation Problem opimises for min(|bTO|) 
                            where O are the Order Conditions of the next higher orders
                            The quadrature Condition is aproximatet
                            
            'none':         Don't use optimisation
    """
    
    #setup Variables for Soulution storage
    p = len(w0) #number of dimentions
    
    uu = np.zeros([p,int(t_final/dt)+100])
    uu[:,0] = w0.copy()
    tt = np.zeros([int(t_final/dt)+100])
    
    
    #Setup Runge Kutta 
    c = rkm.c
    A = rkm.A #has to be lower left triangle
    s = len(c) #number of Stages
    K = np.zeros([p,s])
    
    u = np.array(w0)
    t = 0.
    n = 0
    
    
    #Setup Optimisation Problem
    if objective =='arbitrary':
        O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
        b_op =cp.Variable(s)
        e = np.ones(s) #vector for gola Fnction, just generates the 1-Norm of b
    elif objective == 'L1 of b':
        O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
        ap_op =cp.Variable(s)
        an_op =cp.Variable(s)
        e = np.ones(s) #vector for gola Fnction, just generates the 1-Norm of b
    elif objective == 'Quadrature':
        O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
        ap_op =cp.Variable(s)
        an_op =cp.Variable(s)
        (q,r) = QuadCond(rkm.A,rkm.c,order = rkm.p+1)
        a_op=cp.Variable(s)
        #Idea: seperate |q@b-r| into |q@b - q@a|, set q@a =r and optimise for min|q@b|
    elif objective == 'Hom. Order':
        O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
        ap_op =cp.Variable(s)
        an_op =cp.Variable(s)
        (Q,r) = OrderOpt(rkm.A,rkm.c,order = rkm.p+1)
        q = np.abs(np.ones_like(r)@Q)
    elif objective == 'Order':
        O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
        ap_op =cp.Variable(s)
        an_op =cp.Variable(s)
        (Q,r) = OrderOpt(rkm.A,rkm.c,order = rkm.p+1)
        q = np.abs(np.ones_like(r)@Q)
        (q,r) = QuadCond(rkm.A,rkm.c,order = rkm.p+1)
    elif objective == 'none':
        b_fixed = True
    else:
        print('Objective not known')
        raise InputError
    
        
    #Maybee set up Problem here and treat H as an Paramter
        
    #for debugging b's    
    bb = np.zeros([s,int(t_final/dt)+2])
        
    #print('set up starting to solve')
    
    #Solve ODE
    while t<t_final:
        for i in range(s):
            #compute Stages
            
                
            #K[:,i] = f(t+c[i]*dt,u+dt*K@A[i,:]) 
            #the 0s in A should make shure that no data from an older Step is used
            
            #Maybe better Approach, because A[i,j] = 0 in many places
            u_prime = u.copy()
            for m in range(i):
                u_prime += dt*A[i,m]*K[:,m]
            
            K[:,i] = f(t+c[i]*dt,u_prime)
            
            #print('intermediatestep computed')
        
        if b_fixed == False:
            #Run Optimisation Problem
            if objective =='arbitrary':
                prob = cp.Problem(cp.Minimize(e@b_op),[O@b_op==rhs,u+dt*K@b_op>=0])  
                prob.solve(solver=solver)
                if prob.status != cp.OPTIMAL:
                    print(prob.status)
            
                b = b_op.value
            elif objective == 'L1 of b':
                prob = cp.Problem(cp.Minimize(e@ap_op+e@an_op),
                              [O@(ap_op-an_op+rkm.b)==rhs,u+dt*K@(ap_op-an_op+rkm.b)>=0,ap_op>=0,an_op>=0])  
                prob.solve(solver=solver)
                if prob.status != cp.OPTIMAL:
                    print(prob.status)
            
                b = ap_op.value - an_op.value + rkm.b
            elif objective == 'Quadrature':
                prob = cp.Problem(cp.Minimize(q@ap_op+q@an_op),   
                              [O@(ap_op-an_op-a_op)==rhs,
                               u+dt*K@(ap_op-an_op-a_op)>=0,
                               ap_op>=0,an_op>=0,
                               q@a_op==r]) 
                prob.solve(solver=solver)
                if prob.status != cp.OPTIMAL:
                    print(prob.status)
            
                b = ap_op.value - an_op.value -a_op.value
            elif objective == 'Hom. Order':
                prob = cp.Problem(cp.Minimize(q@ap_op+q@an_op),   
                              [O@(ap_op-an_op)==rhs,u+dt*K@(ap_op-an_op)>=0,ap_op>=0,an_op>=0])  
                #prob.solve(verbose=True)
                prob.solve(solver=solver)
                if prob.status != cp.OPTIMAL:
                    print(prob.status)
            
                b = ap_op.value - an_op.value
            elif objective == 'Order':
                prob = cp.Problem(cp.Minimize(q@ap_op+q@an_op+q@ap_op+q@an_op),   
                              [O@(ap_op-an_op)==rhs,u+dt*K@(ap_op-an_op)>=0,ap_op>=0,an_op>=0])  
                #prob.solve(verbose=True)
                prob.solve(solver=solver)
                if prob.status != cp.OPTIMAL:
                    print(prob.status)
            
                b = ap_op.value - an_op.value
            else:
                raise InputError
            
        else:
            b =rkm.b
        #update
        u += dt*K@b
        n += 1
        t += dt
        
        uu[:,n] = u.copy()
        bb[:,n] = b.copy()
        tt[n] = t
        #print('updated')

        
    return (tt[0:n],uu[:,0:n],bb[:,0:n])
        
        
    

In [ ]:
a = 5 # a>0
#a = 20 #test if we can make the problem more chalanging 
A = np.array([[-a,1],[a,-1]])

print(np.linalg.eig(A))
print(ssp104.real_stability_interval())

In [ ]:
#Testproblem from Kopecz and Meister 2018

def f_lin_I(t,u):
    a = 5 # a>0
    A = np.array([[-a,1],[a,-1]])
    return A@u

u0 =np.array([0.9,0.1])



#stationary solution

 # a>0
A = np.array([[-a,1],[a,-1]])

uoo= np.array([1,a])/(1+a)

#A@uoo

print('uoo:')
print(uoo)

In [ ]:
#Simulate



Objs= {'none':0,'arbitrary':1,'L1 of b':2,'Quadrature':3,'Hom. Order':4,'Order':5}
Results = []


t_final =20

#choose dt with respect to the boundary of the stability region
#EW of System at 6

stab_int = ssp104.real_stability_interval()

#dt=0.1
dt=stab_int/6 -0.1
print(dt)


for obj in Objs:
    print(obj)
    Results.append(RK_variable_b(ssp104,dt,f_lin_I,w0=u0,t_final=t_final,objective = obj,b_fixed=False))



In [ ]:
for obj in Objs:
    i = Objs[obj]
    t,u,b = Results[i]
    plt.plot(t,u[0,:],label=obj)
    plt.plot(t,u[1,:],label=obj)
    

plt.legend()

With bigger stepsizes $\Delta t \geq 1$ oscilations occur at the Homog. Order conditions.
The quadrature condition as objective function leads to an solution that is slowly diverging $u0$
Approx order Cond diverges to $0$ and $1$ for large timesteps $appr. \Delta t = 1.5$
The L1-Norm seems to work fine


Question: which methodes converge and and for which stepsize

Note for futher investigation: for $\Delta t=0.2$ the stationay solution given by the solver has been changed by the Quadrature Condition. This could be explained by the fact that the optimal solutioon is stil a multidimentional space

In [ ]:
tol = 1e-2
for obj in Objs:
    t,u,b = Results[Objs[obj]]
    print(obj)
    if (abs(u[:,-1]-uoo)<tol).all():
        print('yes')
    else:
        print('no')


All converge for smal values (tested at $\Delta t =0.01$)

at $\Delta t =0.1$ arbritary and quadrature don't converges to $u_\infty$

at $\Delta t =1$ only L1 of b and the Order cond converges to $u_\infty$

at $\Delta t =2.2$ (near the border of the stability region) L1 of b converges to $u_\infty$

at $\Delta t = Real stability interval/Eigenvalue \approx 2.32$ the no solution converges $u_\infty$

In [ ]:
t,u,b = Results[Objs['L1 of b']] #'L1 of b' 'Quadrature' 'Hom. Order'
plt.plot(t[1:],b.T[1:,:])


Idea: further investigations of max(b) depending on $\Delta t$? This could give some idea if there is an optimal point for mimimising the truncation error.

Open Question: How does the Quadrature objective behave for an problem with only one degree of Freedom for b? (Maybee) this is per se a bad idea because the feasible region is potentially smaler


Test the order of the Method too make sure that there are no errors in the implementation

In [ ]:
customrk=rk.ExplicitRungeKuttaMethod(A=ssp104.A,b=b[:,2])
customrk.order(tol=1e-10)

Plot the stability region for chosen weights. 

In [ ]:
customrk.plot_stability_region()

In [ ]:
#The standard stability region for comparison
ssp104.plot_stability_region()

The stability region is chnaged drastically by b's choosen by some methods (e.g. 'Quadrature', 'Hom. Order'), even if the b's aren't extremly large. (The 'Hom. Order' and 'Order' methodes yield to b's that aren't very big but stil change the stability region)

# Summary

This notebook highlightet some possible problems and failiure modes of the mehode. This gives some things to keep in mind with te further developement.

It is quite important to keep the b' simmilar to original b's for multiple reasons:

$\bullet$ The Stabilitiy region of the methode can be changed drasticly with the b's

$\bullet$ With large b's the solution of the problem can be changed if $||b||$ is not limited

It seems more fortunade to not primarily reduce the trunation error by trying to infer the Conditions for the next higher order but to enshure that the stability region is still big enough (at least for the ssp104 method). Maybe this is still a reasonable approach for another base method (perhaps some custom method or to fall back form an RK4 to 3rd order) Note: maybe there is a way of changing the b's without changing the stability at important points, which could be enshured by constraints on the b

For this reasons $||b_{optim}-b_{orig}||_1$ seems like a good choice


Only use optimisation problem if needed (also for performance reasons)

In [ ]:
#Tests for the implementation of the quadrature rule as objective

rkm = ssp104
u = 1.
s = 10
K = np.zeros([1,10])
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)
ap_op =cp.Variable(s)
an_op =cp.Variable(s)
(q,r) = QuadCond(rkm.A,rkm.c,order = rkm.p+1)
a_op=cp.Variable(s)
#Idea seperate |q@b-r| into |q@b - q@a|, set q@a =r and optimise for min|q@b|

    
prob = cp.Problem(cp.Minimize(q@ap_op+q@an_op),   
                              [O@(ap_op-an_op-a_op)==rhs,
                               u+dt*K@(ap_op-an_op-a_op)>=0,
                               ap_op>=0,an_op>=0,
                               q@a_op==r])  
                #prob.solve(verbose=True)
prob.solve(solver=cp.ECOS)
if prob.status != cp.OPTIMAL:
    print(prob.status)
            
b = ap_op.value - an_op.value -a_op.value

print(O@b-rhs)